https://www.woman.ru/rest/travel/forum/?sort=new#topic_26

In [1]:
# pip install bs

In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from tqdm.notebook import tqdm
import re

df = pd.DataFrame(columns=['text'])
df

Empty DataFrame
Columns: [text]
Index: []

In [3]:
url = 'https://www.woman.ru/rest/travel/thread/5818807/'

In [4]:
response = requests.get(url)
print(response)

<Response [200]>


In [5]:
html = response.text

In [6]:
# bs = BeautifulSoup(response.text,"lxml")
# print(bs)

In [7]:
url = 'https://www.woman.ru/rest/travel/forum/?sort=new#topic_26'

In [8]:
def get_all_links(html):
    soup = BeautifulSoup(html, features="html.parser")
    links = []
    tds = soup.find_all('div', {'class': 'page__content'})
    for td in tds:
        p = td.find_all('ul', {'class': 'list__container'})
        for i in p:
            q = i.find_all('li', {'class': "list-item"})
            for l in q:
                links.append(l.a['href'])
    return links
get_all_links(get_html(url))

NameError: name 'get_html' is not defined

In [9]:
def get_page_data(html):
    soup = BeautifulSoup(urlopen(str(url.encode('utf-8'))[2:-1]), features="html.parser")

    # user
    tds = soup.find_all('div', {'class':"card__container"})
    users = []
    for td in tds:
        p = td.find_all('div', {'class':"user__maindata"})
        for q in p:
            u = q.find_all('div', {'class':"user__name"})
            for i in u:
                users.append(i.text)
    print(users)


    # time
    tds = soup.find_all('div', {'class':"card__container"})
    times = []
    for td in tds:
        if td.find_all('div', {'class':"card__topic-data"}) == []:
            times.append(td.find('div', {'class':"card__message-data"}).find('time').text)
        else:
            times.append(td.find('div', {'class':"card__topic-data"}).find('time').text)
    print(times)


    # text
    tds = soup.find_all('div', {'class':'card__text'})
    tex = []
    for td in tds: 
        p = td.find_all('p', {'class': 'card__comment'})
        for i in p:
            tex.append(i.text)
    print(tex)
    

    # quote
    tds = soup.find_all('div', {'class':'card__text'})
    temp = []
    for td in tds: 
        q = td.find('p', {'class':'quote__text'})
        if q != None:
            temp.append(q.text)
        else:
            temp.append('')
    print(temp)

    data = ({'user': users,                    
             'time': times,      
             'text': tex,            
             'quote': temp,          
             'topic': ['Depression'] * len(users)
            })          
    return data
get_page_data(get_html(url))

NameError: name 'get_html' is not defined

In [13]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import requests
import json

def get_html(url):
    response = requests.get(url)
    return response.text

def get_all_links(html):
    soup = BeautifulSoup(html, features="html.parser")
    links = []
    tds = soup.find_all('div', {'class': 'page__content'})
    for td in tds:
        p = td.find_all('ul', {'class': 'list__container'})
        for i in p:
            q = i.find_all('li', {'class': "list-item"})
            for l in q:
                links.append(l.a['href'])
    return links

# def get_all_links_on_page(html, link):
#     soup = BeautifulSoup(html, features="html.parser")
#     tds = soup.find_all('tr')
#     for td in tds:
#         i = td.find('td', {'class':'td-block'})
#         if i != None:
#             return(i.find_all('a')[-1].text)
    
def get_links(html, link):
    soup = BeautifulSoup(html, features="html.parser")
    links = []
    tds = soup.find_all('li', {'class':'ipsPagination_page'})
    for td in tds:
        p = td.find_all('a')
        for q in p:
            if link in q['href']:
                links.append(q['href'])
    if len(links) == 0:
        return [link]
    return sorted(list(set(links)))

def get_all_links_one_theme(html):
    soup = BeautifulSoup(html, features="html.parser")
    links = []
    tds = soup.find_all('h4', {'class':'ipsDataItem_title ipsType_break'})
    for td in tds:
        p = td.find_all('a')
        for q in p:
            links.append(q['href'])
    return links

def get_page_data(html):
    soup = BeautifulSoup(urlopen(str(html.encode('utf-8'))[2:-1]), features="html.parser")

    # user
    tds = soup.find_all('div', {'class':"card__container"})
    users = []
    for td in tds:
        p = td.find_all('div', {'class':"user__maindata"})
        for q in p:
            u = q.find_all('div', {'class':"user__name"})
            for i in u:
                users.append(i.text)
#     print(users)


    # time
    tds = soup.find_all('div', {'class':"card__container"})
    times = []
    for td in tds:
        if td.find_all('div', {'class':"card__topic-data"}) == []:
            times.append(td.find('div', {'class':"card__message-data"}).find('time').text)
        else:
            times.append(td.find('div', {'class':"card__topic-data"}).find('time').text)
#     print(times)


    # text
    tds = soup.find_all('div', {'class':'card__text'})
    tex = []
    for td in tds: 
        p = td.find_all('p', {'class': 'card__comment'})
        for i in p:
            tex.append(i.text)
#     print(tex)
    

    # quote
    tds = soup.find_all('div', {'class':'card__text'})
    temp = []
    for td in tds: 
        q = td.find('p', {'class':'quote__text'})
        if q != None:
            temp.append(q.text)
        else:
            temp.append('')
#     print(temp)

    data = ({'user': users,                    
             'time': times,      
             'text': tex,            
             'quote': temp,          
             'topic': ['Travel'] * len(users)
            })          
    return users, times, tex, temp, ['Travel'] * len(users)#data

def page_content(all_links, table):
    for link in tqdm(all_links):
        link = 'https://www.woman.ru' + link
        print(link)
        us, ti, te, tem, l = get_page_data(link)
        for i in range(len(us)):
            if i > 5:
                break
            users.append(us[i])
            times.append(ti[i])
            tex.append(te[i])
            temp.append(tem[i])
            labels.append(l[i])
#         table.append(data)
#     links = get_links(get_html(all_links), all_links)
#     for l in links:
#         all_links_one_theme = get_all_links_one_theme(get_html(l))
#         for t in tqdm(all_links_one_theme):        
#             page = get_links(get_html(t), t)
#             for p in page:
#                 print(p)
#                 data = get_page_data(all_links)
#                 table.append(data)

table = pd.DataFrame(columns={'users', 'times', 'text', 'quote', 'topic'})
users = []
times = []
tex = []
temp = []
labels = []
    
url = 'https://www.woman.ru/rest/travel/forum/?sort=new#topic_26'
all_links = get_all_links(get_html(url))
page_content(all_links, table)

for i in tqdm(range(2, 21)):
    url = 'https://www.woman.ru/rest/travel/forum/' + str(i) + '/?sort=new'
    all_links = get_all_links(get_html(url))
    page_content(all_links, table)
    
table.loc[:, 'users'] = users
table.loc[:, 'times'] = times
table.loc[:, 'text'] = tex
table.loc[:, 'quote'] = temp
table.loc[:, 'topic'] = labels

  0%|          | 0/50 [00:00<?, ?it/s]

https://www.woman.ru/rest/travel/thread/5842499/
https://www.woman.ru/rest/travel/thread/5841882/
https://www.woman.ru/rest/travel/thread/5841854/
https://www.woman.ru/rest/travel/thread/5840986/
https://www.woman.ru/rest/travel/thread/5840919/
https://www.woman.ru/rest/travel/thread/5840755/
https://www.woman.ru/rest/travel/thread/5840509/
https://www.woman.ru/rest/travel/thread/5840161/
https://www.woman.ru/rest/travel/thread/5840115/
https://www.woman.ru/rest/travel/thread/5840111/
https://www.woman.ru/rest/travel/thread/5839845/
https://www.woman.ru/rest/travel/thread/5839199/
https://www.woman.ru/rest/travel/thread/5838942/
https://www.woman.ru/rest/travel/thread/5838883/
https://www.woman.ru/rest/travel/thread/5838869/
https://www.woman.ru/rest/travel/thread/5838868/
https://www.woman.ru/rest/travel/thread/5838761/
https://www.woman.ru/rest/travel/thread/5838649/
https://www.woman.ru/rest/travel/thread/5838182/
https://www.woman.ru/rest/travel/thread/5838246/
https://www.woman.ru

  0%|          | 0/19 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

https://www.woman.ru/rest/travel/thread/5831418/
https://www.woman.ru/rest/travel/thread/5829981/
https://www.woman.ru/rest/travel/thread/5829667/
https://www.woman.ru/rest/travel/thread/5828798/
https://www.woman.ru/rest/travel/thread/5828563/
https://www.woman.ru/rest/travel/thread/5828263/
https://www.woman.ru/rest/travel/thread/5828181/
https://www.woman.ru/rest/travel/thread/5828174/
https://www.woman.ru/rest/travel/thread/5827917/
https://www.woman.ru/rest/travel/thread/5827756/
https://www.woman.ru/rest/travel/thread/5827332/
https://www.woman.ru/rest/travel/thread/5826716/
https://www.woman.ru/rest/travel/thread/5826128/
https://www.woman.ru/rest/travel/thread/5825589/
https://www.woman.ru/rest/travel/thread/5825045/
https://www.woman.ru/rest/travel/thread/5824966/
https://www.woman.ru/rest/travel/thread/5824903/
https://www.woman.ru/rest/travel/thread/5824477/
https://www.woman.ru/rest/travel/thread/5824333/
https://www.woman.ru/rest/travel/thread/5822301/
https://www.woman.ru

...

https://www.woman.ru/rest/travel/thread/5609479/
https://www.woman.ru/rest/travel/thread/5609065/
https://www.woman.ru/rest/travel/thread/5608964/
https://www.woman.ru/rest/travel/thread/5608735/
https://www.woman.ru/rest/travel/thread/5608044/
https://www.woman.ru/rest/travel/thread/5607520/
https://www.woman.ru/rest/travel/thread/5607381/
https://www.woman.ru/rest/travel/thread/5606798/
https://www.woman.ru/rest/travel/thread/5606536/
https://www.woman.ru/rest/travel/thread/5606137/
https://www.woman.ru/rest/travel/thread/5605478/
https://www.woman.ru/rest/travel/thread/5604822/
https://www.woman.ru/rest/travel/thread/5604628/
https://www.woman.ru/rest/travel/thread/5604334/
https://www.woman.ru/rest/travel/thread/5603259/
https://www.woman.ru/rest/travel/thread/5601992/
https://www.woman.ru/rest/travel/thread/5600631/
https://www.woman.ru/rest/travel/thread/5599272/
https://www.woman.ru/rest/travel/thread/5598454/
https://www.woman.ru/rest/travel/thread/5598154/
https://www.woman.ru

  0%|          | 0/50 [00:00<?, ?it/s]

https://www.woman.ru/rest/travel/thread/5591220/
https://www.woman.ru/rest/travel/thread/5591145/
https://www.woman.ru/rest/travel/thread/5590734/
https://www.woman.ru/rest/travel/thread/5590191/
https://www.woman.ru/rest/travel/thread/5589994/
https://www.woman.ru/rest/travel/thread/5589946/
https://www.woman.ru/rest/travel/thread/5589560/
https://www.woman.ru/rest/travel/thread/5589389/
https://www.woman.ru/rest/travel/thread/5589322/
https://www.woman.ru/rest/travel/thread/5588456/
https://www.woman.ru/rest/travel/thread/5585972/
https://www.woman.ru/rest/travel/thread/5585720/
https://www.woman.ru/rest/travel/thread/5585449/
https://www.woman.ru/rest/travel/thread/5584858/
https://www.woman.ru/rest/travel/thread/5584819/
https://www.woman.ru/rest/travel/thread/5584333/
https://www.woman.ru/rest/travel/thread/5584256/
https://www.woman.ru/rest/travel/thread/5584077/
https://www.woman.ru/rest/travel/thread/5583816/
https://www.woman.ru/rest/travel/thread/5582471/
https://www.woman.ru

  0%|          | 0/50 [00:00<?, ?it/s]

https://www.woman.ru/rest/travel/thread/5571478/
https://www.woman.ru/rest/travel/thread/5571466/
https://www.woman.ru/rest/travel/thread/5570863/
https://www.woman.ru/rest/travel/thread/5570341/
https://www.woman.ru/rest/travel/thread/5570217/
https://www.woman.ru/rest/travel/thread/5569582/
https://www.woman.ru/rest/travel/thread/5569357/
https://www.woman.ru/rest/travel/thread/5568912/
https://www.woman.ru/rest/travel/thread/5568909/
https://www.woman.ru/rest/travel/thread/5568771/
https://www.woman.ru/rest/travel/thread/5568730/
https://www.woman.ru/rest/travel/thread/5567724/
https://www.woman.ru/rest/travel/thread/5567273/
https://www.woman.ru/rest/travel/thread/5567072/
https://www.woman.ru/rest/travel/thread/5566843/
https://www.woman.ru/rest/travel/thread/5565821/
https://www.woman.ru/rest/travel/thread/5565659/
https://www.woman.ru/rest/travel/thread/5565064/
https://www.woman.ru/rest/travel/thread/5564979/
https://www.woman.ru/rest/travel/thread/5564972/
https://www.woman.ru

In [11]:
# table.loc[:, 'users'] = users
# table.loc[:, 'times'] = times
# table.loc[:, 'text'] = tex
# table.loc[:, 'quote'] = temp
# table.loc[:, 'topic'] = labels

In [14]:
table

users   topic  \
0                   Гостья   Travel   
1                    Гость   Travel   
2                        🐞   Travel   
3                    Монти   Travel   
4                    Гость   Travel   
...                     ...     ...   
5424            Бухие бухи   Travel   
5425                  Лана   Travel   
5426   шальная императрица   Travel   
5427                 Гость   Travel   
5428                 Эллен   Travel   

                                                   text  \
0     Здравствуйте,планируем в начале мая посетить в...   
1     Возьмите куртку на тонком синтепоне, какой ниб...   
2     Сегодня в футболках даже экстремалы не ходят. ...   
3     Верно, у нас по погоде так и есть, температура...   
4     Добрый день! Сегодня 16, завтра 9, потом 5-6 д...   
...                                                 ...   
5424  Жёсче всех "победа" проверяла, путём погружени...   
5425  Проверяют. В визэйре перед регистрацией есть с...   
5426  основная проверка ручной клади идёт по взгляду...   
5427  Смотря, какая компания. На одном рейсе в Райне...   
5428  Я, из экономии. Самая гадская компания ПОБЕДА....   

                         times quote  
0              Сегодня, 01:55         
1              Сегодня, 07:10         
2              Сегодня, 07:20         
3              Сегодня, 08:47         
4              Сегодня, 11:01         
...                        ...   ...  
5424   22 декабря 2021, 14:54         
5425   22 декабря 2021, 14:54         
5426   22 декабря 2021, 15:16         
5427   22 декабря 2021, 16:25         
5428   23 декабря 2021, 12:27         

[5429 rows x 5 columns]

In [15]:
table.to_csv('Travel_Woman_ru_Copy.csv', index=False)

In [39]:
D = pd.read_csv('Travel_Woman_ru_Copy.csv')
D

users   topic  \
0                   Гостья   Travel   
1                    Гость   Travel   
2                        🐞   Travel   
3                    Монти   Travel   
4                    Гость   Travel   
...                     ...     ...   
5424            Бухие бухи   Travel   
5425                  Лана   Travel   
5426   шальная императрица   Travel   
5427                 Гость   Travel   
5428                 Эллен   Travel   

                                                   text  \
0     Здравствуйте,планируем в начале мая посетить в...   
1     Возьмите куртку на тонком синтепоне, какой ниб...   
2     Сегодня в футболках даже экстремалы не ходят. ...   
3     Верно, у нас по погоде так и есть, температура...   
4     Добрый день! Сегодня 16, завтра 9, потом 5-6 д...   
...                                                 ...   
5424  Жёсче всех "победа" проверяла, путём погружени...   
5425  Проверяют. В визэйре перед регистрацией есть с...   
5426  основная проверка ручной клади идёт по взгляду...   
5427  Смотря, какая компания. На одном рейсе в Райне...   
5428  Я, из экономии. Самая гадская компания ПОБЕДА....   

                         times quote  
0              Сегодня, 01:55    NaN  
1              Сегодня, 07:10    NaN  
2              Сегодня, 07:20    NaN  
3              Сегодня, 08:47    NaN  
4              Сегодня, 11:01    NaN  
...                        ...   ...  
5424   22 декабря 2021, 14:54    NaN  
5425   22 декабря 2021, 14:54    NaN  
5426   22 декабря 2021, 15:16    NaN  
5427   22 декабря 2021, 16:25    NaN  
5428   23 декабря 2021, 12:27    NaN  

[5429 rows x 5 columns]

In [61]:
import re

In [42]:
def cle(time_str):
    if time_str.split()[0][:-1].isalpha():
        return '02 мая 2023'
    else:
        return ' '.join(time_str.split()[:-1])[:-1]

In [52]:
D.loc[:, 'times'] = D.times.apply(lambda x: cle(x))

In [55]:
D.loc[:, 'text'] = D.text.apply(lambda x: re.sub(r'[\n\t\r\xa0]', '', str(x)))

In [57]:
D.loc[:, 'quote'] = D.quote.apply(lambda x: re.sub(r'[\n\t\r\xa0]', '', str(x)))

In [59]:
D.loc[:, 'users'] = D.users.apply(lambda x: re.sub(r'[\n\t\r\xa0]', '', str(x)))

In [60]:
D

users   topic  \
0                   Гостья   Travel   
1                    Гость   Travel   
2                        🐞   Travel   
3                    Монти   Travel   
4                    Гость   Travel   
...                     ...     ...   
5424            Бухие бухи   Travel   
5425                  Лана   Travel   
5426   шальная императрица   Travel   
5427                 Гость   Travel   
5428                 Эллен   Travel   

                                                   text            times quote  
0     Здравствуйте,планируем в начале мая посетить в...      02 мая 2023   nan  
1     Возьмите куртку на тонком синтепоне, какой ниб...      02 мая 2023   nan  
2     Сегодня в футболках даже экстремалы не ходят. ...      02 мая 2023   nan  
3     Верно, у нас по погоде так и есть, температура...      02 мая 2023   nan  
4     Добрый день! Сегодня 16, завтра 9, потом 5-6 д...      02 мая 2023   nan  
...                                                 ...              ...   ...  
5424  Жёсче всех "победа" проверяла, путём погружени...  22 декабря 2021   nan  
5425  Проверяют. В визэйре перед регистрацией есть с...  22 декабря 2021   nan  
5426  основная проверка ручной клади идёт по взгляду...  22 декабря 2021   nan  
5427  Смотря, какая компания. На одном рейсе в Райне...  22 декабря 2021   nan  
5428  Я, из экономии. Самая гадская компания ПОБЕДА....  23 декабря 2021   nan  

[5429 rows x 5 columns]

In [62]:
D.to_csv('Travel.csv', index=False)

In [63]:
D = pd.read_csv('Travel.csv')
D

users   topic  \
0                   Гостья   Travel   
1                    Гость   Travel   
2                        🐞   Travel   
3                    Монти   Travel   
4                    Гость   Travel   
...                     ...     ...   
5424            Бухие бухи   Travel   
5425                  Лана   Travel   
5426   шальная императрица   Travel   
5427                 Гость   Travel   
5428                 Эллен   Travel   

                                                   text            times quote  
0     Здравствуйте,планируем в начале мая посетить в...      02 мая 2023   NaN  
1     Возьмите куртку на тонком синтепоне, какой ниб...      02 мая 2023   NaN  
2     Сегодня в футболках даже экстремалы не ходят. ...      02 мая 2023   NaN  
3     Верно, у нас по погоде так и есть, температура...      02 мая 2023   NaN  
4     Добрый день! Сегодня 16, завтра 9, потом 5-6 д...      02 мая 2023   NaN  
...                                                 ...              ...   ...  
5424  Жёсче всех "победа" проверяла, путём погружени...  22 декабря 2021   NaN  
5425  Проверяют. В визэйре перед регистрацией есть с...  22 декабря 2021   NaN  
5426  основная проверка ручной клади идёт по взгляду...  22 декабря 2021   NaN  
5427  Смотря, какая компания. На одном рейсе в Райне...  22 декабря 2021   NaN  
5428  Я, из экономии. Самая гадская компания ПОБЕДА....  23 декабря 2021   NaN  

[5429 rows x 5 columns]

In [ ]:
import random
import numpy as np
ind = np.random.choice(len(D), 50, False)
ind

In [ ]:
X = []
for i in ind:
    X.append(D['text'][i])

In [ ]:
count = 0 # счетчик хороших записей

In [ ]:
X[0]

In [ ]:
count+=1
print(count)

In [ ]:
42/(50)

In [8]:
count = 0
for i in range(len(D)):
    count += len(D['user'][i].split())
print('Количество записей с одного сайта по одной теме:', count)

Количество записей с одного сайта по одной теме: 7829


In [13]:
import random
import numpy as np
ind = np.random.choice(len(D), 20, False)
ind

array([ 79,  93, 115, 100,  56, 102,   1, 116, 103, 104, 143,  19,   7,
       151,  62, 128, 127,  10,  97,  92])

In [14]:
X = []
for i in ind:
    X.append(np.random.choice(D['text'][i].split(sep=']'), 5, False))

In [36]:
X[19]

array([", ['Что именно чушь?'",
       ', [\'Автор! Вам всё понятно? Или это была "разводка"?\'',
       ', [\'Если бы вы были хоть немного образованным человеком, то писали бы слово Россия, а не Раша. Кстати, что такое "раша"? Вы сможете дать грамотное пояснение? Жду ваших объяснений. Не сомневаюсь, что вы перейдете на грубость, либо сдерживаясь, ваш гнев и агрессия уйдут внутрь, а так люди обычно получают заболевания психосоматической природы.\'',
       ", ['Вообще-то смешно как раз вы написали. Как раз по Мир нормальный курс. Менять только если валюту, но тратить сейчас наличную валюту на поездку в РБ как по мне тупо.'",
       ''], dtype='<U1976')

In [ ]:
count = 0 # счетчик хороших записей

In [ ]:
count+=1
print(count)

In [37]:
2+2+2+2+2+3+2+2+3+1+3+3+2+4+4+3+2+3+4+1

50

In [40]:
50/(100-20)

0.625

In [15]:
D = pd.read_csv('Travel_woman_talks.csv')
D

user  \
0              ['\nГость', '\nJasmiine', '\nGolubka']   
1   ['\nДарина', '\nМиленькая)', '\nЛаванда', '\nL...   
2   ['\nМарта', '\nXyhuvupu', '\nOlgaOlga_5', '\nT...   
3   ['\nEkaterinaN', '\nLolaa', '\nAlivia', '\nКюш...   
4   ['\nGolubka', '\nbaranets20-20', '\nGolubka', ...   
5   ['\nAlexa', '\nNissa', '\nМикра', '\nAlexa', '...   
6              ['\nikari', '\ncorenad', '\nKetaryKa']   
7   ['\nKatosha', '\nАнтонина89', '\nГалина Марков...   
8   ['\nБьянка-Янка', '\nМайская', '\nAleksandra11...   
9                                      ['\nVenezian']   
10          ['\nVladlena Petrova', '\nMaria Glushko']   
11  ['\nsienna', '\nГость Тетя Зиля', '\nГость FiF...   
12                   ['\nTeam2Travel', '\nZadorovVW']   
13  ['\nGalina1579', '\nZhenya212', '\npetrenko_ta...   
14                  ['\nTeam2Travel', '\nAlekceuBak']   
15  ['\nСтервочка)', '\nМультяшка', '\nМадам', '\n...   
16  ['\nAugustina', '\nАдам Малхиев', '\nwomantalk...   
17  ['\nantonovivan', '\nИнна-88', '\nvdowencko', ...   
18                                        ['\nГость']   
19                          ['\nГость', '\nКюшечкаН']   
20  ['\nNatali_Nikolaevna', '\nppolik', '\nКюшечкаН']   
21                             ['\nLyudmila_Pavlova']   
22                                      ['\nЛисичка']   
23                             ['\nГость', '\nKonst']   
24  ['\nSvetlo', '\nMariAnt', '\nДжессика', '\nChr...   
25  ['\nГость', '\ndjumba', '\nВероника В', '\nTwi...   
26  ['\nАлена Миронова', '\nВермоер', '\nSabri1', ...   
27                                  ['\nTeam2Travel']   
28                 ['\nГость Анютка', '\nИнтересная']   
29  ['\nГрета', '\nTalita', '\nBizzara', '\nXuos',...   
30  ['\nDiana', '\nGalo4ka', 'Еще одни насущный во...   
31  ['\nОльга1329', '\nAugustina', '\nВероника В',...   
32                                  ['\nTeam2Travel']   
33                                  ['\nTeam2Travel']   
34  ['\nHelga', '\nТамерлана', '\nТамерлана', '\nH...   
35  ['\nФиона', '\nritokn', '\nlianohka', '\nBally...   
36  ['\nГость Галочка', '\nRealmccoy', '\nTalita',...   
37  ['\nSindy', '\nkarina.28.28', '\nsandalik17@ma...   
38  ['\nLapochka', '\nRealmccoy', '\nprostomary', ...   
39                          ['\nMulya', '\nKurochka']   

                                                 time  \
0   ['\nОпубликовано: 21 мар 2019\n', '\nОпубликов...   
1   ['\nОпубликовано: 18 апр 2016\n', '\nОпубликов...   
2   ['\nОпубликовано: 13 мар 2017\n', '\nОпубликов...   
3   ['\nОпубликовано: 23 апр 2019\n', '\nОпубликов...   
4   ['\nОпубликовано: 30 янв 2021\n', '\nОпубликов...   
5   ['\nОпубликовано: 2 мар 2016\n', '\nОпубликова...   
6   ['\nОпубликовано: 3 авг 2020\n', '\nОпубликова...   
7   ['\nОпубликовано: 12 фев 2020\n', '\nОпубликов...   
8   ['\nОпубликовано: 28 май 2016\n', '\nОпубликов...   
9                   ['\nОпубликовано: 17 дек 2019\n']   
10  ['\nОпубликовано: 23 окт 2019\n', '\nОпубликов...   
11  ['\nОпубликовано: 11 сен 2017\n', '\nОпубликов...   
12  ['\nОпубликовано: 10 окт 2017\n', '\nОпубликов...   
13  ['\nОпубликовано: 5 дек 2018\n\t\t\t\n\t\t\t\t...   
14  ['\nОпубликовано: 27 ноя 2017\n', '\nОпубликов...   
15  ['\nОпубликовано: 10 май 2016\n', '\nОпубликов...   
16  ['\nОпубликовано: 23 дек 2016\n', '\nОпубликов...   
17  ['\nОпубликовано: 20 июл 2016\n', '\nОпубликов...   
18                  ['\nОпубликовано: 21 мар 2019\n']   
19  ['\nОпубликовано: 14 ноя 2018\n', '\nОпубликов...   
20  ['\nОпубликовано: 21 ноя 2018\n', '\nОпубликов...   
21                  ['\nОпубликовано: 11 дек 2018\n']   
22                  ['\nОпубликовано: 10 дек 2018\n']   
23  ['\nОпубликовано: 14 ноя 2018\n', '\nОпубликов...   
24  ['\nОпубликовано: 8 июн 2016\n', '\nОпубликова...   
25  ['\nОпубликовано: 5 апр 2017\n', '\nОпубликова...   
26  ['\nОпубликовано: 5 мар 2018\n\t\t\t\n\t\t\t\t...   
27                  ['\nОпубликовано: 22 янв 2018\n']   
28  ['\nОпубликовано: 23 ноя 2017\n', '\nОпубликов...   
2

In [16]:
count = 0
for i in range(len(D)):
    count += len(D['user'][i].split())
print('Количество записей с одного сайта по одной теме:', count)

Количество записей с одного сайта по одной теме: 415


In [63]:
[x for x in D.iloc[39].text.split(sep = '],')][0].split(sep="\', ")[0]# 38

"['\\n\\n\\tВсем добрый день! Кто отдыхал в Подмосковье, подскажите хороший дом отдыха или тур базу!! Очень нужно! Отдых планируется на выходные в июле, желательно что была большая территория где можно кататься на велосипедах, ну и конечно желательно наличие водоема! Спасибо огромное!!!\\n\\n"

In [64]:
38/40

0.95

In [66]:
D = pd.read_csv('Travel_Woman_ru.csv')
D

user  \
0    [' Автор ', ' Гость ', ' Гость ', ' Деловунка ...   
1    [' Автор ', ' Засранец ', ' Гость ', ' Гость '...   
2    [' Автор ', ' Гость ', ' Гость ', ' Гость ', '...   
3    [' Автор ', ' AngryCat ', ' Гость ', ' Гость '...   
4    [' Мария ', ' Гость ', ' Гость ', ' Мария ', '...   
..                                                 ...   
995  [' TRumkina01 ', ' Гость ', ' Гость ', ' TRumk...   
996  [' Автор ', ' Мурлыка ', ' Гость ', ' Дина ', ...   
997  [' Автор ', ' Гость ', ' Гость ', ' Гость ', '...   
998                  [' Автор ', ' Гость ', ' Гость ']   
999  [' Елена ', ' Гость ', ' Елена ', ' Гость ', '...   

                                                  time  \
0    [' Сегодня, 15:04 ', ' Сегодня, 15:40 ', ' Сег...   
1    [' 25 марта, 16:22 ', ' 25 марта, 16:56 ', ' 2...   
2    [' 25 марта, 16:12 ', ' 25 марта, 16:42 ', ' 2...   
3    [' 25 марта, 15:58 ', ' 25 марта, 16:06 ', ' 2...   
4    [' 25 марта, 10:01 ', ' 25 марта, 10:29 ', ' 2...   
..                                                 ...   
995  [' 31 октября 2021, 19:33 ', ' 31 октября 2021...   
996  [' 30 октября 2021, 22:37 ', ' 30 октября 2021...   
997  [' 30 октября 2021, 14:42 ', ' 30 октября 2021...   
998  [' 28 октября 2021, 20:11 ', ' 29 октября 2021...   
999  [' 27 октября 2021, 20:50 ', ' 27 октября 2021...   

                                                  text  \
0    ['Сейчас сложно получить визу в Европу и дорог...   
1    ['Научный руководитель настаивает на стажировк...   
2    ['кто из Тюмени расскажите о своём городе, плю...   
3    ['Срочно понадобилось ехать из Екб в Самару, б...   
4    ['Голдлайн плюс? (Сибутрамин)Добрый день! Соби...   
..                                                 ...   
995  ['Добрый всем день!Кто может подсказать, где б...   
996  ['Хотели бы там побывать? Я да, но боюсь змей,...   
997  ['Ни разу не была в тех местах. Посоветуйте от...   
998  ['Мужчина оказался на удивление спокойным чело...   
999  ['Главное, конечно, чтобы олинклюзив был вкусн...   

                                                 quote  \
0                                     ['', '', '', '']   
1    ['', '', '', 'О , бы сгонялТакая возможность. ...   
2    ['', '', '', 'Знаю что это богатый нефтяной ре...   
3    ['', '', '', '', '', '', '', 'А вообще вот :С ...   
4    ['', '', '', 'ты больная? возьми туда парацета...   
..                                                 ...   
995  ['', '', '', 'А вы что хотите? Культурный досу...   
996  ['', '', '', '', '', '', 'Крис Хэмсфорт, кенгу...   
997                       ['', '', '', '', '', '', '']   
998                            ['', '', 'о чем речь?']   
999  ['', '', 'Много иностранцев в шамс сафага, сей...   

                                                 topic  
0             ['Travel', 'Travel', 'Travel', 'Travel']  
1    ['Travel', 'Travel', 'Travel', 'Travel', 'Trav...  
2    ['Travel', 'Travel', 'Travel', 'Travel', 'Trav...  
3    ['Travel', 'Travel', 'Travel', 'Travel', 'Trav...  
4    ['Travel', 'Travel', 'Travel', 'Travel', 'Trav...  
..                                                 ...  
995  ['Travel', 'Travel', 'Travel', 'Travel', 'Trav...  
996  ['Travel', 'Travel', 'Travel', 'Travel', 'Trav...  
997  ['Travel', 'Travel', 'Travel', 'Travel', 'Trav...  
998                     ['Travel', 'Travel', 'Travel']  
999  ['Travel', 'Travel', 'Travel', 'Travel', 'Trav...  

[1000 rows x 5 columns]

In [67]:
count = 0
for i in range(len(D)):
    count += len(D['user'][i].split())
print('Количество записей с одного сайта по одной теме:', count)

Количество записей с одного сайта по одной теме: 47764


In [97]:
[x for x in D.iloc[25].text.split(sep = '],')][0].split(sep="\', ")[0]# 22

"['Всем привет. Расскажите ваши мысли по поводу жизни во Франции, было бы интересно послушать. Сейчас изучаю французский, английский знаю почти в совершенстве. Работаю как самозанятая. Планирую через 2-3 года переехать во Францию для получения второго высшего или магистра, а потом в аспирантуру. Хочу работать преподавателем истории. Знаю, там очень большая бумажная волокита, митинги, высокие цены на жилье, но я практически ничего не знаю про менталитет самих французов. Из знакомых у меня там всего 2 человека.По поводу поступления не волнуюсь, у меня идеальные оценки, принимаю участие во множестве мероприятий и тд и тп. Просто меня триггерит именно сам момент проживания/общения/культурного обмена."